In [1]:
# News Source : https://www.forbes.com/sites/digital-assets/2024/10/25/what-crypto-investors-should-expect-from-a-pro-crypto-congress/

In [2]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB

In [3]:
import os
import openai
import numpy as np
import pandas as pd
import json
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding

In [4]:
openai.api_key = "NULL"

In [5]:
System_Prompt = """You are a professional news article summarizer, equipped to distill complex information into clear, concise summaries. Your goal is to extract essential details and present them in a structured manner. Follow these steps:

Step 1: Read and Analyze the Article Thoroughly
- Read the entire article carefully to grasp the context, main points, and supporting details.
- Focus on the 5Ws (Who, What, When, Where, Why) and the How, identifying the main event or issue along with key people, organizations, locations, and relevant dates.

Step 2: Extract Key Elements for the Summary
- Main Event or Topic: Identify the central event, development, or issue discussed in the article.
- Context: Provide background information or circumstances surrounding the main event.
- Key Figures: Highlight important individuals, groups, or organizations involved.
- Quotes and Evidence: Select impactful quotes or pieces of evidence that reinforce the article's message.
- Future Implications: Consider any consequences, future actions, or developments related to the event.

Step 3: Structure the Summary
The summary should be concise yet informative, following this structured format:
- Headline: Create a brief, compelling headline (5-10 words) that encapsulates the essence of the article.
- Lead (1-2 sentences): Offer a brief introduction summarizing the main event or topic, covering the ‘What’ and ‘Who.’
- Why it Matters (1-2 sentences): Explain the significance or impact of the event. Why should the reader care?
- Details (2-3 sentences): Include key points, evidence, quotes, or relevant background information. Ensure to address ‘When’ and ‘Where.’
- Zoom in (1-2 sentences): Explore a specific aspect or perspective from the article that adds depth, such as a notable quote or unique viewpoint.
- Flashback (1 sentence): Provide a quick historical reference or a look back at related past events for context.
- Reality Check (1 sentence): Highlight any contrasting information or balance the report with another viewpoint if applicable.
- Conclusion (1 sentence): Wrap up with a sentence summarizing potential future actions, outcomes, or implications.

Step 4: Maintain Objectivity and Neutrality
- Ensure the summary is free from bias or personal opinions. Present information factually and clearly.
- Use a professional and accessible tone, making the summary understandable to all readers, regardless of their familiarity with the topic.

Step 5: Format and Review the Summary
- Review the summary to ensure logical flow, error-free content, and accurate representation of key points.
- Verify the length of each section is appropriate, keeping each segment brief while ensuring nothing critical is omitted.

Once you have processed the article using these steps, present the summary in the specified format."""

In [6]:
user_message = """
Crypto has made headlines as the 2024 Presidential Election enters the final hectic stretch, but that has overshadowed the wide-ranging influence that crypto lobbyists and lobbying dollars will have on the next Congress. With over $160 million spent by the crypto industry in the current election cycle, investors and policymakers alike should prepare for a bipartisan influx of crypto-friendly lawmakers across both chambers of Congress. With multiple candidates poised to ascend to seats in the House of Representatives this voting block is set to add a significant voice in support of an industry that been continuously on the defensive under the Biden Administration and SEC chair Gary Gensler.

Interestingly enough the spending by the crypto industry has occurred on a bipartisan basis, including millions of dollars spent at the primary level to ensure pro-crypto candidates appeared on the general election ballot in the first place. Since January 2023 crypto super PACs have funded advertising campaigns involving 67 candidates, including ad campaigns in support of the candidates in question as well as more negative campaigns targeted at opponents of the pro-crypto individual. Given the rapid ascent of the crypto lobbying industry – that although not yet ranked #1 in total dollars spent - has certainly made crypto a talking point and issue of this election cycle from a nearly invisible starting point. Given that, investors and policymakers alike should be aware of what a quantifiably more pro-crypto Congress could mean going forward.

A New SEC Chairperson
One of the few things that politicians on both sides of the aisle can agree upon is that the SEC regime that has been led by Gary Gensler has taken an antagonistic approach to the crypto sector. After a track record of drafting cutting edge regulation in a prior role at the CFTC, as well as developing and delivering courses on blockchain and tokenized assets at MIT, the attitude taken by the chair has been quite the opposite of prior public comments. Whether it be the regulation-by-enforcement tactic taken in the form of multiple lawsuits and legal notices against token issuers and crypto exchanges, or the combative comments made during public appearances the message has been clear. Crypto has been listed and treated as a detriment to public markets rather than the innovative solution it truly represents.

Flush with dollars from crypto super PACs and other lobbyists, the new Congress (and potentially the White House) – will face both pressure and expectations to install a new SEC head. While technically there is no obligation for an incumbent chairperson to resign upon a change in either Congressional or White House leadership that has been the case previously. One thing that crypto investors and advocates should be wary of, however, is that (like under Gensler) even an apparently pro-crypto individual can undertake an aggressive and antagonistic campaign.

Legislation: Stablecoin Or Bitcoin
Another expectation that policymakers and investors should come into 2025 with is that – after years of false starts and seemingly interminable hearings and testimonials – some kind of crypto related legislation is due to be passed into law. While there have been multiple efforts in the past to codify the treatment and classification of stablecoins, there are two primary reasons why the new Congress will more strongly incentivized to do so. First the reality that TradFi institutions across the board have entered into the crypto arena with an array of products and services has made crypto and crypto-adjacent topics more understandable and less frightening for the non-expert consumer. Second is the growing profile of technologies and technology applications – including crypto alongside the red hot AI sector – in every day conversation. Consumers increasingly understand that tools exist that will enable daily transactions and activities to be performed in a more cost and time efficient manner. Consumer interest translates to voter expectations, especially as the private sector continues to innovate and produce solutions.
"""

In [7]:
struct = [{'role' : 'system', 'content' : System_Prompt}]
struct.append({"role": "user", "content": user_message})
chat = openai.ChatCompletion.create(model="gpt-4o-mini", messages = struct)
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})
print("Assistant:", response)

Assistant: **Headline:** Crypto Lobbying Surge Influences 2024 Elections and Future Legislation

**Lead:** As the 2024 Presidential Election intensifies, the crypto industry's significant lobbying efforts, exceeding $160 million, highlight the growing influence of pro-crypto lawmakers in Congress.

**Why it Matters:** This surge in lobbying dollars could reshape U.S. policy towards cryptocurrencies, providing a more favorable environment for a sector that has faced regulatory challenges under the current administration.

**Details:** The crypto industry has funded campaigns for 67 candidates since January 2023, promoting pro-crypto candidates while targeting opponents. With multiple candidates poised to support the industry in the House of Representatives, the crypto lobby’s impact has positioned cryptocurrency as a noteworthy issue in the election cycle. 

**Zoom in:** Despite the positive momentum, stakeholders should remain cautious. Multiple lawsuits and an enforcement-heavy approa